In [1]:
from pymilvus import MilvusClient

MILVUS_URI="http://127.0.0.1:19530"
MILVUS_TOKEN="root:Milvus"




async def get_milvus_client(uri: str = MILVUS_URI, token: str = MILVUS_TOKEN, db_name: str = None):
    """
    Returns a MilvusClient instance.
    
    If db_name is provided, connects to that database.
    If not, connects using default database.
    """
    try:
        if db_name:
            return MilvusClient(uri=uri, token=token, db_name=db_name)
        return MilvusClient(uri=uri, token=token)
    except Exception as e:
        print(f"Failed to connect to Milvus (db_name={db_name}): {e}")

In [2]:
milvus_client=await get_milvus_client()

In [6]:
milvus_client.create_database("kapture")

In [ ]:
dir(milvus_client)

In [ ]:
if milvus_client.drop_database('iud'):
    print("hg")
else:
    print("hdddb")

In [ ]:
milvus_client.list_databases()

In [ ]:
milvus_client.list_collections()

In [ ]:
%cd ..

In [ ]:
milvus_client.list_collections()

In [ ]:
milvus_client.create_database("kapture")

In [ ]:
import re
from datetime import datetime

def generate_safe_collection_name(base: str) -> str:
    # Keep only letters and numbers
    safe_base = re.sub(r'[^a-zA-Z0-9]', '', base)
    # Append a timestamp to ensure uniqueness (also only numbers)
    timestamp = datetime.utcnow().strftime('%Y%m%d%H%M%S')
    return f"{safe_base}{timestamp}"


In [ ]:
collection_name = generate_safe_collection_name("123")

In [ ]:
milvus_client.list_collections()

In [ ]:
milvus_client.drop_collection("kapture_22220250618124613")

In [3]:
%cd ..

/home/fariq.rahman/work/Auto_QA_Components/AUTOQA_RAG/app


In [4]:
from services.mongodb_client import get_mongodb_connection

/home/fariq.rahman/miniconda3/envs/RAG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
_, db = get_mongodb_connection()


In [7]:
db

AsyncIOMotorDatabase(Database(MongoClient(host='mongodb+srv://cluster0.9dg51lt.mongodb.net', document_class=dict, tz_aware=False, connect=False, retrywrites=True, w='majority', appname='Cluster0', driver=DriverInfo(name='Motor', version='3.7.1', platform='asyncio')), 'AUTOQA_RAG'))

In [ ]:
# @router.post("/web_link_to_knowledge", summary="Create a knowledge base from a web link",
#     description="""
#     Crawls the provided web link, scrapes content, and stores it in Milvus.

#     - `name`: The name of the knowldeg 
#     - `cmd_id`: unique cmd id for each client
#     - `link`: The link to crawl/scrape
#     """)
# async def Making_Knowledge_Base(payload: LinkToKnowledge):
# # async def dump_knowledge_base_info(cmd_id,name,link,status,up_time,error=None):

#     knowledge_base_id=generate_safe_collection_name(payload.cmd_id)
#     await dump_knowledge_base_info(cmd_id=payload.cmd_id,name=payload.name,link=payload.link,know_base_id=knowledge_base_id)

#     web_scraping_data = await crawl_website(payload.link)
#     print("<<<<<--- Web Scraping Completed ---->>>>>>>>")
#     extracted_json = await extract_pages_to_json(web_scraping_data)
#     print("<<<<<--- Text formated Completed ---->>>>>>>>")
#     text = await all_content_formatting(extracted_json)
#     chunked_text = await Recursive_chunking(text=text)
#     print("<<<<<--- Chunked   Completed ---->>>>>>>>")

#     try:
#         collection_ready = await Create_Collection(
#             collection_name=knowledge_base_id,
#             description=f"Web data for {payload.link}"
#         )

#         if not collection_ready:
#             return {"error": "Failed to create or access collection"}
        
#         inserted = await insert_data_to_collection(
#             collection_name=knowledge_base_id,
#             chunked_data=chunked_text
#         )
#         if not inserted:
#             return {"error": "Failed to insert data into Milvus collection"}
#         return {
#             "message": "Ingestion complete",
#             "chunks_stored": len(chunked_text),
#             "collection":knowledge_base_id ,
#             "database": "kapture"
#         }
    

#     except Exception as e:
#         logging.exception("Failed during ingestion")
#         return {"error": str(e)}


In [ ]:
async def dump_questions(questions,client_id,test_id,section_id,error=None):
    _, db = get_mongodb_connection()
    document={
        "_id" : str(uuid.uuid4()) + str(datetime.datetime.now().timestamp()),
        "client_id":client_id,
        "test_id":test_id,
        "section_id":section_id,
        "document_name" : "AutoCoahing_documents",
        "added_at": datetime.datetime.now().isoformat(),
        "questions":questions,
        "error": error
        }
    await db["Question_Creation"].insert_one(document)

In [ ]:
await dump_section_data(data=None, client_id=payload.client_id, test_id=payload.test_id,link=None, error=f"ParseError: {str(pe)}")

In [ ]:
from mongodb.client import get_mongodb_connection
from fastapi import APIRouter , HTTPException

async def section_retrieval_for_question_gen(client_id, test_id, section_id):
    _, db = get_mongodb_connection()

    doc = await db["Document_Sections"].find_one(
        {
            "client_id": client_id,
            "test_id": test_id,
            "sections.id": section_id
        },
        {
            "sections": {"$elemMatch": {"id": section_id}},
            "_id": 0
        }
    )

    if not doc or "sections" not in doc or not doc["sections"]:
        raise HTTPException(status_code=404, detail="Section not found")

    return doc["sections"][0]


async def section_retrival_for_conversation(client_id, test_id, section_id):
    _, db = get_mongodb_connection()

    doc = await db["Document_Sections"].find_one(
        {
            "client_id": client_id,
            "test_id": test_id,
            "sections.id": section_id
        },
        {
            "sections": {"$elemMatch": {"id": section_id}},
            "_id": 0
        }
    )
    
    if doc["sections"][0]['is_conversation'] == 'false':
        raise HTTPException(status_code=404, detail="The Section is not elibile for creating conversation")

    return doc["sections"][0]

In [8]:
import io
import re
import requests
import pdfplumber
import logging
from typing import List

async def download_extract_text_from_pdf(links: List[str]) -> str:
    """
    Download each PDF, skip failures, extract only “real” text pages,
    strip out (cid:…) junk, and drop pages whose text is <50% letters.
    Returns one big string of all cleaned text.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "application/pdf",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "identity",
        "Referer": "https://google.com"
    }

    cleaned_pages = []

    for url in links:
        if not url.lower().endswith(".pdf"):
            logging.warning(f"Skipping non‑PDF link: {url}")
            continue

        try:
            resp = requests.get(url, headers=headers, timeout=(10, 60))
            resp.raise_for_status()
        except Exception as e:
            logging.warning(f"{url} download failed: {e}")
            continue

        if "pdf" not in resp.headers.get("Content-Type", "").lower():
            logging.warning(f"Not a PDF MIME‑type: {url}")
            continue

        try:
            with pdfplumber.open(io.BytesIO(resp.content)) as pdf:
                for page in pdf.pages:
                    raw = page.extract_text() or ""
                    if not raw.strip():
                        continue

                    # 1) remove (cid:###)
                    no_cid = re.sub(r"\(cid:\d+\)", "", raw)
                    # 2) drop non‑printable
                    printable = "".join(ch for ch in no_cid if ch.isprintable())
                    # 3) compute letter ratio
                    total = len(printable)
                    letters = sum(ch.isalpha() for ch in printable)
                    if total == 0 or letters/total < 0.5:
                        # too much gibberish—skip this page
                        continue

                    # page is “real text”
                    cleaned_pages.append(printable.strip())

        except Exception as e:
            logging.warning(f"Failed to parse PDF {url}: {e}")
            continue

    # join all kept pages
    return "\n\n".join(cleaned_pages)


In [9]:
links = [
    "https://www.jiobank.in/docs/grievance-redressal/Integrated-Ombudsman-Scheme.pdf",
    "https://some-other-site.com/valid-pdf.pdf",
    "https://www.jiobank.in/docs/banking-outlets/list-of-banking-outlets.pdf"
]
text = await download_extract_text_from_pdf(links)
print(text)


In [4]:
from firecrawl import FirecrawlApp
from typing import List
app = FirecrawlApp(api_key="fc-60797ab4662d46d8b16dee063cb2a649")
def crawl_multiple(urls: List[str], limit_each: int = 1):
    all_results = []
    for u in urls:
        print(f"📡 Crawling {u}")
        docs = app.batch_scrape_urls(u, limit=limit_each, formats=['markdown'])
        all_results.extend(docs.data)
    return all_results


In [9]:
# Scrape multiple websites:
batch_scrape_result = app.batch_scrape_urls(['firecrawl.dev', 'mendable.ai'], formats=['markdown'])
print(batch_scrape_result)

success=True status='completed' completed=0 total=0 creditsUsed=0 expiresAt=datetime.datetime(2025, 6, 20, 6, 13, 3, tzinfo=TzInfo(UTC)) next=None data=[]


In [11]:
batch_scrape_result

CrawlStatusResponse(success=True, status='completed', completed=0, total=0, creditsUsed=0, expiresAt=datetime.datetime(2025, 6, 20, 6, 13, 3, tzinfo=TzInfo(UTC)), next=None, data=[])

## Multiple_URL

In [16]:
from firecrawl import FirecrawlApp, ScrapeOptions
from typing import List, Dict

# 1️⃣ Initialize your app
app = FirecrawlApp(api_key="fc-60797ab4662d46d8b16dee063cb2a649")

def scrape_multiple_urls(
    urls: List[str],
    formats: List[str] = ["markdown"],
    timeout_ms: int = 120_000
) -> List[Dict]:

    batch_result = app.batch_scrape_urls(
        urls,
        formats=formats,
        timeout=timeout_ms
    )
    
    return batch_result.data

if __name__ == "__main__":
    urls_to_scrape = [
        "https://firecrawl.dev/",
        "https://www.kapture.cx/",
        "https://docs.firecrawl.dev/learn/rag-llama3",
    ]
    
    scraped_pages = scrape_multiple_urls(urls_to_scrape)
    



In [15]:
scraped_pages

[FirecrawlDocument(url=None, markdown='[Introducing SearchSearch the web & scrape the results with one API call.](https://docs.firecrawl.dev/features/search)\n\n[💥 Get 2 months free with yearly plan](https://www.firecrawl.dev/pricing)\n\n# Turn websites into   LLM-ready data\n\nPower your AI apps with clean data crawled from any website. [It\'s also open source.](https://github.com/mendableai/firecrawl)\n\nhttps://example.com\n\nStart for free (500 credits)Start for free\n\n200 Response\n\n\\[\\\n\\\n{\\\n\\\n"url": "https://example.com",\\\n\\\n"markdown": "# Getting Started...",\\\n\\\n"json": {\\\n"title": "Guide",\\\n"docs": ...\\\n},\\\n\\\n"screenshot": "https://example.com/hero.png",\\\n\\\n}\\\n\\\n...\\\n\\\n\\]\n\n## Trusted by Top Companies\n\n[![Customer Logo](https://www.firecrawl.dev/logos/zapier.png)](https://www.zapier.com/)\n\n[![Customer Logo](https://www.firecrawl.dev/logos/nvidia-com.png)](https://www.nvidia.com/)\n\n[![Customer Logo](https://www.firecrawl.dev/logos

In [21]:
import markdown as md
from bs4 import BeautifulSoup
import json
import re
def extract_pages_to_json(docs: List[object]) -> json:
    """
    From a list of FirecrawlDocument-like objects, writes a JSON list where each entry has:
      - text: cleaned, tag-free, link-free
      - file_links: [.pdf/.docx/etc URLs]
      - image_links: [all image URLs from <img>]
    """
    # Define which extensions count as “files”
    doc_exts = ('.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx')
    url_strip = re.compile(r'https?://\S+')
    # print(str(docs)[:100])
    output: List[Dict[str, object]] = []
    for doc in docs:
        # 1) Get HTML source
        if getattr(doc, "rawHtml", None):
            html = doc.rawHtml
        elif getattr(doc, "html", None):
            html = doc.html
        else:
            md_src = getattr(doc, "markdown", "") or ""
            html = md.markdown(md_src)

        soup = BeautifulSoup(html, "html.parser")

        # 2) Find and classify all <a href> links
        file_links = []
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if href.lower().endswith(doc_exts):
                file_links.append(href)

        # 3) Find all <img> src URLs
        image_links = [img["src"] for img in soup.find_all("img", src=True)]

        # 4) Extract clean text
        text = soup.get_text(separator="\n")
        text = url_strip.sub("", text)           # remove any leftover URLs
        text = re.sub(r"\n{3,}", "\n\n", text).strip()

        output.append({
            "text": text,
            "file_links": file_links,
            "image_links": image_links
        })

    # # 5) Write JSON
    # with open(output_path, "w", encoding="utf-8") as f:
    #     json.dump(output, f, ensure_ascii=False, indent=2)
    return output

In [24]:
def unique_pdf(links:List[str],available:List[str])->List[str]:
    checked=[]
    for link in links:
        if link not in available:
            checked.append(link)
    return checked


In [27]:
def download_extract_text_from_pdf(links: List[str]) -> str:
    """
    Download each PDF, skip failures, extract only “real” text pages,
    strip out (cid:…) junk, and drop pages whose text is <50% letters.
    Returns one big string of all cleaned text.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "application/pdf",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "identity",
        "Referer": "https://google.com"
    }

    cleaned_pages = []

    for url in links:
        if not url.lower().endswith(".pdf"):
            logging.warning(f"Skipping non‑PDF link: {url}")
            continue

        try:
            resp = requests.get(url, headers=headers, timeout=(10, 60))
            resp.raise_for_status()
        except Exception as e:
            logging.warning(f"{url} download failed: {e}")
            continue

        if "pdf" not in resp.headers.get("Content-Type", "").lower():
            logging.warning(f"Not a PDF MIME‑type: {url}")
            continue

        try:
            with pdfplumber.open(io.BytesIO(resp.content)) as pdf:
                for page in pdf.pages:
                    raw = page.extract_text() or ""
                    if not raw.strip():
                        continue

                    # 1) remove (cid:###)
                    no_cid = re.sub(r"\(cid:\d+\)", "", raw)
                    # 2) drop non‑printable
                    printable = "".join(ch for ch in no_cid if ch.isprintable())
                    # 3) compute letter ratio
                    total = len(printable)
                    letters = sum(ch.isalpha() for ch in printable)
                    if total == 0 or letters/total < 0.5:
                        # too much gibberish—skip this page
                        continue

                    # page is “real text”
                    cleaned_pages.append(printable.strip())

        except Exception as e:
            logging.warning(f"Failed to parse PDF {url}: {e}")
            continue

    # join all kept pages
    return "\n\n".join(cleaned_pages)





def all_content_formatting(extracted:json) -> str:
    '''
        It will accept the extracted information from wesite and format everythink
    '''
    all_text:str=""
    used_pdf=[]
    for field in extracted:
        all_text+=field['text']
        new_links= unique_pdf(field['file_links'],used_pdf)
        used_pdf.extend(new_links)
        content_from_pdf= download_extract_text_from_pdf(new_links)
        all_text+="\n"+content_from_pdf+"\n"
    return all_text


In [28]:
data=extract_pages_to_json(scraped_pages)

In [29]:
testx=all_content_formatting(data)

In [30]:
testx

'Introducing SearchSearch the web & scrape the results with one API call.\n\n💥 Get 2 months free with yearly plan\n\nTurn websites into   LLM-ready data\n\nPower your AI apps with clean data crawled from any website. \nIt\'s also open source.\n\nStart for free (500 credits)Start for free\n\n200 Response\n\n[\\\n\\\n{\\\n\\\n"url": "\n\\\n"markdown": "# Getting Started...",\\\n\\\n"json": {\\\n"title": "Guide",\\\n"docs": ...\\\n},\\\n\\\n"screenshot": "\n\\\n}\\\n\\\n...\\\n\\\n]\n\nTrusted by Top Companies\n\nDeveloper first\n\nStart scraping tonight\n\nEnhance your apps with industry leading web scraping and crawling capabilities\n\nScrape\n\nGet llm-ready data from websites. Markdown, JSON, screenshot, etc\n\nCrawl\n\nCrawl all the pages on a website and get data for each page\n\nSearch\n\nNew\n\nSearch the web and get full content from results\n\n1\n\n2\n\n3\n\n4\n\n5\n\n6\n\n7\n\n8\n\n```\n// npm install @mendable/firecrawl-js\n\nimport FirecrawlApp from \'@mendable/firecrawl-js\'

## MULTI-Query RAG

In [ ]:
from typing import List
def generate_multiple_query(query:str) -> List[str]:
    system_prompt="""Generate 5 similar prompts to broaden the context of the following prompt:\n{What re the products available in bigbasket
            output_format
            {
            querys:[quer1,query2..]
            }
            """
    
    

In [7]:
system_prompt="""Generate 5 similar prompts to broaden the context of the following prompt:\n{What re the products available in bigbasket
output_format
{
querys:[quer1,query2..]
}
"""

In [ ]:
from fastapi import APIRouter, HTTPException
from typing import List, Dict
from config import GEMINI_API_KEY, GEMINI_MODEL, embedding_model, cross_encoder, milvus_client
from google import genai
from google.genai import types
from rank_bm25 import BM25Okapi
import json

router = APIRouter()
client = genai.Client(api_key=GEMINI_API_KEY)

# System prompt to generate sub-queries
Subquery_prompt = '''
You are a query generator. Given a user question, generate exactly five concise, related search queries that explore different facets of the original question. Output as a JSON array of strings.
'''

async def generate_subqueries(query: str) -> List[str]:
    payload = client.models.generate_content(
        model=GEMINI_MODEL,
        contents=query,
        config=types.GenerateContentConfig(
            system_instruction=Subquery_prompt,
            response_mime_type="application/json"
        )
    )
    try:
        subqs = json.loads(payload.text)
        if not isinstance(subqs, list) or len(subqs) != 5:
            raise ValueError("Invalid subqueries format")
        return subqs
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Subquery generation failed: {e}")

async def vector_search(collection: str, query: str, top_k: int = 2) -> List[str]:
    q_emb = embedding_model.encode([query]).tolist()[0]
    results = milvus_client.search(
        collection_name=collection,
        anns_field="embedding",
        data=[q_emb],
        limit=top_k,
        search_params={"metric_type": "COSINE", "params": {"ef": 200}},
        output_fields=["text"]
    )
    return [hit[0]["entity"]["text"] for hit in results]

async def keyword_search(query: str, texts: List[str], token_corpus: List[List[str]], top_k: int = 2) -> List[str]:
    bm25 = BM25Okapi(token_corpus)
    scores = bm25.get_scores(query.split())
    ranked = sorted(zip(texts, scores), key=lambda x: x[1], reverse=True)
    return [txt for txt, _ in ranked[:top_k]]

async def fetch_all_docs(collection: str, batch_size: int = 500):
    docs, offset = [], 0
    while True:
        rows = milvus_client.query(
            collection_name=collection,
            filter="",
            output_fields=["text", "keyword_text"],
            limit=batch_size,
            offset=offset
        )
        if not rows:
            break
        docs.extend(rows)
        offset += len(rows)
        if len(rows) < batch_size:
            break
    texts, tokens = [], []
    for row in docs:
        texts.append(row["text"])
        kt = row.get("keyword_text", "").
        if isinstance(kt, str):
            try:
                kt = json.loads(kt)
            except:
                kt = kt.split()
        tokens.append(kt)
    return texts, tokens

async def hybrid_retrieve_top2(collection: str, query: str) -> List[str]:
    texts, tokens = await fetch_all_docs(collection)
    kw = await keyword_search(query, texts, tokens, top_k=2)
    vec = await vector_search(collection, query, top_k=2)
    # merge and rerank
    seen, candidates = set(), []
    for txt in kw + vec:
        if txt not in seen:
            seen.add(txt)
            candidates.append(txt)
    ce_scores = cross_encoder.predict([(query, txt) for txt in candidates])
    reranked = sorted(zip(candidates, ce_scores), key=lambda x: x[1], reverse=True)
    return [text for text, _ in reranked[:2]]

@router.post("/multi_hybrid_search", summary="Multi-query RAG retrieval", description="Generate 5 subqueries and retrieve 2 chunks each via hybrid search.")
async def multi_hybrid_search(payload: Retrival) -> Dict:
    try:
        subqueries = await generate_subqueries(payload.query)
        all_chunks = []
        retrieval_map = {}
        for subq in subqueries:
            chunks = await hybrid_retrieve_top2(payload.coll, subq)
            retrieval_map[subq] = chunks
n           all_chunks.extend(chunks)
        # Prepare RAG input
        rag_input = {"Question": payload.query, "Retrieved": retrieval_map}
        rag_response = await RAG_Answering(retrived_information=rag_input)
        return {"subqueries": subqueries, "retrievals": retrieval_map, "rag": rag_response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [3]:
%cd ..

/home/fariq.rahman/work/Auto_QA_Components/AUTOQA_RAG/app


In [4]:
from fastapi import APIRouter, HTTPException
from typing import List, Dict
from config import GEMINI_API_KEY, GEMINI_MODEL, embedding_model, cross_encoder, milvus_client
from google import genai
from google.genai import types
from rank_bm25 import BM25Okapi
import json

/home/fariq.rahman/miniconda3/envs/RAG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
client = genai.Client(api_key=GEMINI_API_KEY)


In [14]:
Subquery_prompt = '''
You are a query generator. Given a user question, generate exactly five concise, related search queries that explore different facets of the original question. Output as a JSON array of strings.
'''

In [15]:
async def generate_subqueries(query: str) -> List[str]:
    payload = client.models.generate_content(
        model=GEMINI_MODEL,
        contents=query,
        config=types.GenerateContentConfig(
            system_instruction=Subquery_prompt,
            response_mime_type="application/json"
        )
    )
    try:
        subqs = json.loads(payload.text)
        if not isinstance(subqs, list) or len(subqs) != 5:
            raise ValueError("Invalid subqueries format")
        return subqs
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Subquery generation failed: {e}")

In [16]:
li = await generate_subqueries('what are the daily products available in BIGBASKET')

In [17]:
li

['BigBasket daily essentials list',
 'Common grocery items BigBasket',
 'BigBasket fresh produce availability',
 'Household products on BigBasket',
 'Daily use items BigBasket categories']

In [34]:
from services.retrival import vector_search,keyword_search,fetch_all_docs,hybrid_retrieve

from pydantic import BaseModel,Field
class Retrival(BaseModel):
    coll:str=Field(...,description="The name of the collection")
    query:str=Field(...,description="The User Query")

In [35]:
async def hybrid_retrieve_top2(collection: str, query: str) -> List[str]:
    texts, tokens = await fetch_all_docs(collection)
    kw = await keyword_search(query, texts, tokens, top_k=2)
    vec = await vector_search(collection, query, top_k=2)
    # merge and rerank
    seen, candidates = set(), []
    for txt in kw + vec:
        if txt not in seen:
            seen.add(txt)
            candidates.append(txt)
    ce_scores = cross_encoder.predict([(query, txt) for txt in candidates])
    reranked = sorted(zip(candidates, ce_scores), key=lambda x: x[1], reverse=True)
    return [text for text, _ in reranked[:2]]

In [46]:
async def multi_hybrid_search(coll,query) -> Dict:
    try:
        subqueries = await generate_subqueries(query)
        all_chunks = []
        retrieval_map = {}
        for subq in subqueries:
            chunks = await hybrid_retrieve(collection_name=coll,query= subq,top_k=2)
            retrieval_map[subq] = chunks
        all_chunks.extend(chunks)
        # Prepare RAG input
        rag_input = {"Question": query, "Retrieved": retrieval_map}
    #     rag_response = await RAG_Answering(retrived_information=rag_input)
    #     return {"subqueries": subqueries, "retrievals": retrieval_map, "rag": rag_response}
        return rag_input
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [47]:
milvus_client.list_collections()

['kapture_840020250619120119',
 'kapture_840020250619122818',
 'kapture_840020250619123850',
 'kapture_840020250619124215',
 'kapture_840020250619114922',
 'kapture_840020250619115336',
 'kapture_840020250619120633',
 'kapture_840020250619121727',
 'kapture_840020250619123555']

In [48]:
data=await multi_hybrid_search("kapture_840020250619120119","How to create the accout in jio bank")

In [50]:
data['Retrieved']

{'How to open Jio Payments Bank account': ["You can open an account with us through\n\nJioFinance App\n\nMyJio App\n\nVisit any of our banking outlets : \n\nDo I need a Jio number in order to open the bank account?\n\nNo. You can open it with any number. The number has to be linked with your Aadhar card.\n\nCan I open an Jio bank account without a smartphone?\n\nNo. You don't need a smartphone to operate a Jio Payments Bank account. You can bank with us at any of our outlets.\n\nWhat documents do I need to open an Jio bank account?",
  'Hassle-free process\n\nEnjoy paperless onboarding.\n\nNo hidden charges\n\nCompletely transparent process.\n\nSeamless transactions\n\nFaster payments for you.\n\nSecure\n\nSafer digital transactions.\n\nGuaranteed savings\n\n2.5% interest on savings account.\n\nStill have questions?\n\nWho is eligible for opening an account with Jio Payments Bank?\n\nAny Indian resident above 18 years of age can open a Jio Payments Bank account.\n\nHow do I open a savi

## RAG WITH PDF

In [61]:
import json
from typing import List, Dict 
import markdown as md
from bs4 import BeautifulSoup
import re
import io
import requests
import pdfplumber
import logging

In [102]:
async def download_extract_text_from_pdf(links: List[str]) -> str:
    """
    Download each PDF, skip failures, extract only “real” text pages,
    strip out (cid:…) junk, and drop pages whose text is <50% letters.
    Returns one big string of all cleaned text.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "application/pdf",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "identity",
        "Referer": "https://google.com"
    }

    cleaned_pages = []

    for url in links:
        if not url.lower().endswith(".pdf"):
            logging.warning(f"Skipping non‑PDF link: {url}")
            continue

        try:
            resp = requests.get(url, headers=headers, timeout=(10, 60))
            resp.raise_for_status()
        except Exception as e:
            logging.warning(f"{url} download failed: {e}")
            continue

        if "pdf" not in resp.headers.get("Content-Type", "").lower():
            logging.warning(f"Not a PDF MIME‑type: {url}")
            continue

        try:
            with pdfplumber.open(io.BytesIO(resp.content)) as pdf:
                for page in pdf.pages:
                    raw = page.extract_text() or ""
                    if not raw.strip():
                        continue

                    # 1) remove (cid:###)
                    no_cid = re.sub(r"\(cid:\d+\)", "", raw)
                    # 2) drop non‑printable
                    printable = "".join(ch for ch in no_cid)
                    # 3) compute letter ratio
                    total = len(printable)
                    print("<<<<<<<<<<<<<<<<<<<<<<< TOTAL>>>>>>>>>>>>>>>>>>>>>")
                    print(total)
                    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>")
                    letters = sum(ch.isalpha() for ch in printable)
                    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<< letters>>>>>>>>>>>>>>>>>>>>")
                    print(letters)
                    if total == 0 or letters/total < 0.5:
                        # too much gibberish—skip this page
                        print("<<<<<<<<<<<<< NOT ELIGIBLE>>>>>>>>>>>>>>>>>>>")
                        print(page)
                        print(raw)
                        continue

                    # page is “real text”
                    cleaned_pages.append(printable.strip())

        except Exception as e:
            logging.warning(f"Failed to parse PDF {url}: {e}")
            continue

    # join all kept pages
    return "\n\n".join(cleaned_pages)


In [103]:
data_=await download_extract_text_from_pdf(["https://kapture-p-v2.storage.googleapis.com/1000227/ticket/0625/174884917532402ca74y5j8y/lc-sop-manual.pdf"])

<<<<<<<<<<<<<<<<<<<<<<< TOTAL>>>>>>>>>>>>>>>>>>>>>
3560
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<< letters>>>>>>>>>>>>>>>>>>>>
570
<<<<<<<<<<<<< NOT ELIGIBLE>>>>>>>>>>>>>>>>>>>
<Page:1>
LIVE CHAT AGENT STANDARD OPERATING PROCEDURE
ESD SOP # 1
Live Chat Division Revision # 2
Implementation Date 04.16.2021
Page # 1 of 17 Last Reviewed/Updated 04.15.2021
SOP Owner T. Granillo Approval 04.15.2021
Contents
Purpose................................................................................................................................................... 2
Scope ...................................................................................................................................................... 2
Prerequisites ........................................................................................................................................... 2
Responsibilities ...............................................................

<<<<<<<<<<<<<<<<<<<<<<< TOTAL>>>>>>>>>>>>>>>>>>>>>
2437
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<< letters>>>>>>>>>>>>>>>>>>>>
1897
<<<<<<<<<<<<<<<<<<<<<<< TOTAL>>>>>>>>>>>>>>>>>>>>>
2120
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<< letters>>>>>>>>>>>>>>>>>>>>
1679
<<<<<<<<<<<<<<<<<<<<<<< TOTAL>>>>>>>>>>>>>>>>>>>>>
2148
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<<< letters>>>>>>>>>>>>>>>>>>>>
1705


In [99]:
print(data_)


LIVE CHAT AGENT STANDARD OPERATING PROCEDURE
LIVE CHAT AGENT STANDARD OPERATING PROCEDURE
Purpose
To create a standard and uniform process for Live Chat agents and anyone operating in Live Chat moving
forward.
Upon the implementation of a dedicated Live Chat department, it was determined that a process of
standardization had not been implemented. It is the goal of the Live Chat department to produce
consistent service to our customers.
Live Chat is meant to be a virtual resource equal to that provided by our OneStop’s and WorkSource
centers. To encourage customer engagement an umbrella of uniformity in: how the service is delivered,
the information provided, and the experience by the customer needs to be fulfilled. The procedure
manual is meant to service as the guiding principle in this endeavor.
Scope
This procedure manual is meant for any person that has access to, or intends to access, the Velaro
software for the purpose of providing remote virtual service to WorkSource WA customer

In [ ]:
"".isprintable()

False

In [84]:
string1 = "Hello World"
print(string1.isprintable())  # Output: True

string2 = "Hello\nWorld"
print(string2.isprintable())  # Output: False

string3 = ""
print(string3.isprintable())  # Output: True

True
False
True
